In [ ]:
# Import necessary libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import random


# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
# Define model paths
knight_model_path = '/content/gdrive/My Drive/Datasets/knight_detector_saved_model'
rook_model_path = '/content/gdrive/My Drive/Datasets/rook_detector_saved_model'
bishop_model_path = '/content/gdrive/My Drive/Datasets/bishop_detector_saved_model'
queen_model_path = '/content/gdrive/My Drive/Datasets/queen_detector_saved_model'
king_model_path = '/content/gdrive/My Drive/Datasets/king_detector_saved_model'
pawn_model_path = '/content/gdrive/My Drive/Datasets/pawn_detector_saved_model'


In [ ]:
# Load models
knight_model = load_model(knight_model_path)
rook_model = load_model(rook_model_path)
bishop_model = load_model(bishop_model_path)
queen_model = load_model(queen_model_path)
king_model = load_model(king_model_path)
pawn_model = load_model(pawn_model_path)


#calibrated_binary_model = joblib.load('/content/gdrive/My Drive/Datasets/knight_detector_calibrated_model.pkl')

In [ ]:
def preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array


In [ ]:
model_weights = {
    'Knight': 0.1908,
    'Bishop': 0.1622,
    'Rook': 0.1635,
    'Queen': 0.1506,
    'King': 0.1607,
    'Pawn': 0.1722
}




def predict_chess_piece(img_path):
    processed_image = preprocess_image(img_path)

    # Get predictions from each model
    knight_pred = knight_model.predict(processed_image)
    print("Knight Prediction:", knight_pred)
    bishop_pred = bishop_model.predict(processed_image)
    print("Bishop Prediction:", bishop_pred)
    rook_pred = rook_model.predict(processed_image)
    print("Rook Prediction:", rook_pred)
    queen_pred = queen_model.predict(processed_image)
    print("Queen Prediction:", queen_pred)
    king_pred = king_model.predict(processed_image)
    print("King Prediction:", king_pred)
    pawn_pred = pawn_model.predict(processed_image)
    print("Pawn Prediction:", pawn_pred)

    # Apply weights to the predictions
    weighted_predictions = np.array([
        knight_pred * model_weights['Knight'],
        bishop_pred * model_weights['Bishop'],
        rook_pred * model_weights['Rook'],
        queen_pred * model_weights['Queen'],
        king_pred * model_weights['King'],
        pawn_pred * model_weights['Pawn']
    ])

    # Determine the final prediction
    predicted_piece_index = np.argmax(weighted_predictions.sum(axis=0))
    pieces = ["Knight", "Bishop", "Rook", "Queen", "King", "Pawn"]
    return pieces[predicted_piece_index]


In [ ]:
# Function to Randomly Select Images
categories = ["Knight", "Rook", "Bishop", "Queen", "King", "Pawn"]
dataset_dir = '/content/gdrive/My Drive/Datasets/Chess_Vision'

def get_random_images(num_images=5):
    selected_images = []
    for category in categories:
        category_path = os.path.join(dataset_dir, category)
        images = os.listdir(category_path)
        num_to_sample = min(len(images), num_images)  # Sample up to the available number
        selected_images.extend(random.sample([(os.path.join(category_path, img), category) for img in images], num_to_sample))
    return selected_images


# Testing Function
def test_random_images(num_images):
    random_images = get_random_images(num_images)
    correct_predictions = 0

    for img_path, true_label in random_images:
        predicted_label = predict_chess_piece(img_path)
        print(f"Image: {img_path}, True Label: {true_label}, Predicted Label: {predicted_label}")
        if predicted_label == true_label:
            correct_predictions += 1

    accuracy = correct_predictions / (num_images * 6)
    return accuracy



In [ ]:
known_images = [
    ("/content/gdrive/My Drive/Datasets/Chess_Vision/Knight/00000190.jpg", "Knight"),
    ("/content/gdrive/My Drive/Datasets/Chess_Vision/Pawn/00000199.jpg", "Pawn"),
    # Add more images and their true labels
]

for img_path, true_label in known_images:
    predicted_label = predict_chess_piece(img_path)
    print(f"Image: {img_path}, True Label: {true_label}, Predicted Label: {predicted_label}")


1/1 [==============================] - 2s 2s/step
Knight Prediction: [[0.00221035]]
1/1 [==============================] - 1s 1s/step
Bishop Prediction: [[0.9752566]]
1/1 [==============================] - 1s 1s/step
Rook Prediction: [[0.00671336]]
1/1 [==============================] - 1s 974ms/step
Queen Prediction: [[0.11606818]]


1/1 [==============================] - 1s 728ms/step
King Prediction: [[0.989821]]


1/1 [==============================] - 1s 709ms/step
Pawn Prediction: [[0.04673212]]
Image: /content/gdrive/My Drive/Datasets/Chess_Vision/Knight/00000190.jpg, True Label: Knight, Predicted Label: Knight
1/1 [==============================] - 1s 532ms/step
Knight Prediction: [[0.9987946]]
1/1 [==============================] - 1s 548ms/step
Bishop Prediction: [[0.55481386]]
1/1 [==============================] - 1s 594ms/step
Rook Prediction: [[0.14208758]]
1/1 [==============================] - 1s 582ms/step
Queen Prediction: [[0.30714574]]
1/1 [==============================] - 1s 596ms/step
King Prediction: [[0.44536817]]
1/1 [==============================] - 1s 562ms/step
Pawn Prediction: [[0.97517294]]
Image: /content/gdrive/My Drive/Datasets/Chess_Vision/Pawn/00000199.jpg, True Label: Pawn, Predicted Label: Knight


In [ ]:
def test_individual_model(model, test_images, expected_label):
    correct_predictions = 0

    for img_path in test_images:
        processed_image = preprocess_image(img_path)
        prediction = model.predict(processed_image)
        predicted_label = "Yes" if prediction[0][0] > 0.5 else "No"

        if (predicted_label == expected_label):
            correct_predictions += 1

        print(f"Image: {img_path}, Predicted Label: {predicted_label}")

    accuracy = correct_predictions / 2
    print(f"Accuracy for {expected_label}: {accuracy * 100}%")

# Example usage - test knight model
knight_test_images = ["/content/gdrive/My Drive/Datasets/Chess_Vision/Knight/00000190.jpg", "/content/gdrive/My Drive/Datasets/Chess_Vision/Knight/00000197.jpg"]
bishop_test_images = ["/content/gdrive/My Drive/Datasets/Chess_Vision/Bishop/00000199.jpg", "/content/gdrive/My Drive/Datasets/Chess_Vision/Bishop/00000192.jpg"]
test_individual_model(bishop_model, bishop_test_images, "Yes")
rook_test_images = ["/content/gdrive/My Drive/Datasets/Chess_Vision/Rook/00000189.jpg", "/content/gdrive/My Drive/Datasets/Chess_Vision/Rook/00000185.jpg"]
test_individual_model(rook_model, rook_test_images, "Yes")
queen_test_images = ["/content/gdrive/My Drive/Datasets/Chess_Vision/Queen/00000279.jpg", "/content/gdrive/My Drive/Datasets/Chess_Vision/Queen/00000219.jpg"]
test_individual_model(queen_model, queen_test_images, "Yes")
king_test_images = ["/content/gdrive/My Drive/Datasets/Chess_Vision/King/00000167.jpg", "/content/gdrive/My Drive/Datasets/Chess_Vision/King/00000175.jpg"]
test_individual_model(king_model, king_test_images, "Yes")
pawn_test_images = ["/content/gdrive/My Drive/Datasets/Chess_Vision/Pawn/00000139.jpg", "/content/gdrive/My Drive/Datasets/Chess_Vision/Pawn/00000140.jpg"]
test_individual_model(pawn_model, pawn_test_images, "Yes")

1/1 [==============================] - 1s 896ms/step
Image: /content/gdrive/My Drive/Datasets/Chess_Vision/Knight/00000190.jpg, Predicted Label: No
1/1 [==============================] - 1s 534ms/step
Image: /content/gdrive/My Drive/Datasets/Chess_Vision/Knight/00000197.jpg, Predicted Label: No
Accuracy for Yes: 0.0%
1/1 [==============================] - 1s 538ms/step
Image: /content/gdrive/My Drive/Datasets/Chess_Vision/Bishop/00000199.jpg, Predicted Label: No
1/1 [==============================] - 1s 551ms/step
Image: /content/gdrive/My Drive/Datasets/Chess_Vision/Bishop/00000192.jpg, Predicted Label: No
Accuracy for Yes: 0.0%
1/1 [==============================] - 1s 557ms/step
Image: /content/gdrive/My Drive/Datasets/Chess_Vision/Rook/00000189.jpg, Predicted Label: Yes
1/1 [==============================] - 1s 546ms/step
Image: /content/gdrive/My Drive/Datasets/Chess_Vision/Rook/00000185.jpg, Predicted Label: Yes
Accuracy for Yes: 100.0%
1/1 [==============================] - 1s 5

In [ ]:
# Example usage
accuracy = test_random_images(1)
print(f"Accuracy on 100 random images: {accuracy * 100}%")

1/1 [==============================] - 1s 550ms/step
Knight Prediction: [[0.04918201]]
1/1 [==============================] - 1s 541ms/step
Bishop Prediction: [[0.76670754]]
1/1 [==============================] - 1s 558ms/step
Rook Prediction: [[0.02955765]]
1/1 [==============================] - 1s 543ms/step
Queen Prediction: [[0.10941649]]
1/1 [==============================] - 1s 563ms/step
King Prediction: [[0.96038264]]
1/1 [==============================] - 1s 543ms/step
Pawn Prediction: [[0.05448903]]
Image: /content/gdrive/My Drive/Datasets/Chess_Vision/Knight/00000092.jpg, True Label: Knight, Predicted Label: Knight
1/1 [==============================] - 1s 527ms/step
Knight Prediction: [[0.99997276]]
1/1 [==============================] - 1s 554ms/step
Bishop Prediction: [[0.9943134]]
1/1 [==============================] - 1s 531ms/step
Rook Prediction: [[0.9993575]]
1/1 [==============================] - 1s 600ms/step
Queen Prediction: [[0.32857105]]
1/1 [==================